# Dataset Profiling - LATAM Data Engineer Challenge

## Objetivo

Este notebook realiza un análisis exhaustivo del dataset **antes** de implementar las soluciones a las preguntas q1, q2 y q3.

**¿Por qué hacer profiling primero?**

1. **Identificar casos borde** que podrían romper las implementaciones
2. **Validar calidad de datos** y detectar registros inválidos
3. **Guiar decisiones de diseño** basadas en evidencia, no suposiciones
4. **Estimar tiempos de procesamiento** para elegir estrategias time vs memory
5. **Evitar retrabajos** por descubrir problemas tarde

## Metodología

- **Streaming**: No cargamos el dataset completo en memoria
- **Muestreo inteligente**: Analizamos todo el dataset cuando es posible
- **Métricas accionables**: Cada hallazgo se traduce en una decisión técnica

---

In [16]:
# Imports
import orjson
import pandas as pd
import emoji
from pathlib import Path
from collections import defaultdict, Counter
from datetime import datetime
import time

print("Dependencies loaded successfully")

Dependencies loaded successfully


## 1. Carga y Validación Inicial

Primero verificamos que el dataset esté disponible y en formato JSON Lines correcto.

In [17]:
# Configuración
DATASET_PATH = "../../data/raw/farmers-protest-tweets-2021-2-4.json"
SAMPLE_SIZE = None  # None = todo el dataset, N = primeras N líneas

dataset_path = Path(DATASET_PATH)

if not dataset_path.exists():
    print(f"ERROR: Dataset not found at {DATASET_PATH}")
    print("\nDownload the dataset first:")
    print("  python src/dataset/download_dataset.py")
else:
    file_size_mb = dataset_path.stat().st_size / (1024 * 1024)
    print(f"Dataset found: {file_size_mb:.2f} MB")

Dataset found: 388.83 MB


In [18]:
# Conteo rápido de líneas totales
print("Counting total lines...")

total_lines = 0
start_time = time.time()

with open(dataset_path, 'rb') as f:
    for line in f:
        total_lines += 1

elapsed = time.time() - start_time

print(f"\nTotal lines: {total_lines:,}")
print(f"Counting time: {elapsed:.2f} seconds")
print(f"Throughput: {total_lines/elapsed:.0f} lines/second")

Counting total lines...

Total lines: 117,407
Counting time: 0.28 seconds
Throughput: 413520 lines/second


## 2. Análisis de Esquema y Calidad

Este análisis se divide en dos partes:

**Parte 1: Estructura y Esquema**
- Análisis rápido de los primeros 1,000 tweets
- Identificación de todos los campos presentes
- Tipos de datos por campo
- Ejemplo de estructura completa

**Parte 2: Análisis Estadístico Completo**
- Procesamiento de todo el dataset
- Campos faltantes o null
- Tipos de datos incorrectos
- Errores de parseo JSON
- Métricas de calidad de datos

In [19]:
# PART 1: Schema and Structure Analysis
print("="*60)
print("  SCHEMA AND STRUCTURE ANALYSIS")
print("="*60)
print("\nAnalyzing first 1,000 tweets to understand data structure...\n")

schema_stats = {
    'all_fields': set(),
    'field_types': defaultdict(Counter),
    'sample_tweets': [],
    'lines_analyzed': 0,
    'non_dict_count': 0,
}

SCHEMA_SAMPLE_SIZE = 1000

with open(dataset_path, 'rb') as f:
    for line_num, line in enumerate(f, 1):
        if line_num > SCHEMA_SAMPLE_SIZE:
            break
        
        try:
            tweet = orjson.loads(line)
            
            # Verificar que sea un diccionario
            if not isinstance(tweet, dict):
                schema_stats['non_dict_count'] += 1
                continue
            
            schema_stats['lines_analyzed'] += 1
            
            # Guardar primeros 3 tweets como muestra
            if len(schema_stats['sample_tweets']) < 3:
                schema_stats['sample_tweets'].append(tweet)
            
            # Recolectar todos los campos
            for field in tweet.keys():
                schema_stats['all_fields'].add(field)
                
                # Registrar tipo de dato
                field_value = tweet[field]
                field_type = type(field_value).__name__
                schema_stats['field_types'][field][field_type] += 1
                
        except orjson.JSONDecodeError:
            continue

print(f"Lines analyzed: {schema_stats['lines_analyzed']:,}")
print(f"Non-dictionary lines found: {schema_stats['non_dict_count']}")
print()

# Mostrar todos los campos encontrados
print("All fields found in tweets:")
for field in sorted(schema_stats['all_fields']):
    types = schema_stats['field_types'][field]
    type_str = ', '.join([f"{t}({c})" for t, c in types.most_common()])
    print(f"  - {field}: {type_str}")

print()

# Mostrar ejemplo de estructura completa
if schema_stats['sample_tweets']:
    print("Example tweet structure:")
    print("-" * 60)
    import json
    sample = schema_stats['sample_tweets'][0]
    print(json.dumps(sample, indent=2, ensure_ascii=False)[:1000] + "...")


  SCHEMA AND STRUCTURE ANALYSIS

Analyzing first 1,000 tweets to understand data structure...

Lines analyzed: 1,000
Non-dictionary lines found: 0

All fields found in tweets:
  - content: str(1000)
  - conversationId: int(1000)
  - date: str(1000)
  - id: int(1000)
  - lang: str(1000)
  - likeCount: int(1000)
  - media: NoneType(735), list(265)
  - mentionedUsers: NoneType(709), list(291)
  - outlinks: list(1000)
  - quoteCount: int(1000)
  - quotedTweet: NoneType(620), dict(380)
  - renderedContent: str(1000)
  - replyCount: int(1000)
  - retweetCount: int(1000)
  - retweetedTweet: NoneType(1000)
  - source: str(1000)
  - sourceLabel: str(990), NoneType(10)
  - sourceUrl: str(990), NoneType(10)
  - tcooutlinks: list(1000)
  - url: str(1000)
  - user: dict(1000)

Example tweet structure:
------------------------------------------------------------
{
  "url": "https://twitter.com/ArjunSinghPanam/status/1364506249291784198",
  "date": "2021-02-24T09:23:35+00:00",
  "content": "The world

### Análisis de esquema (muestra inicial)

Se analizaron **1.000 tweets** para validar la estructura del dataset antes de implementar las soluciones.

- Todas las líneas son JSON válidas, no se encontraron registros corruptos.
- El esquema es consistente entre tweets.
- Los campos clave (`date`, `content`, `user`) están presentes en el 100% de la muestra.
- `mentionedUsers` es opcional (≈71% `None`, ≈29% lista), por lo que debe manejarse explícitamente.
- Otros campos como `media` y `quotedTweet` también pueden ser `None`.

**Implicaciones:** el dataset puede procesarse en streaming sin limpieza previa, usando `mentionedUsers` como fuente principal para menciones y parseando `date` directamente en formato ISO 8601.


In [20]:
# PART 2: Complete Statistical Analysis
print("\n" + "="*60)
print("  COMPLETE STATISTICAL ANALYSIS")
print("="*60)
print("\nAnalyzing all tweets for detailed statistics...\n")

stats = {
    'valid_lines': 0,
    'invalid_lines': 0,
    'non_dict_lines': 0,
    'parse_errors': [],
    'missing_fields': defaultdict(int),
    'date_formats': Counter(),
    'content_lengths': [],
    'mentions_distribution': [],
    'emoji_count': 0,
    'tweets_with_emojis': 0,
    'sample_tweets': [],
}

mentions_stats = {
    'with_mentions': 0,
    'without_mentions': 0,
    'null_mentions': 0,
    'empty_list': 0,
}

start_time = time.time()

with open(dataset_path, 'rb') as f:
    for line_num, line in enumerate(f, 1):
        
        # Progreso cada 10k líneas
        if line_num % 10000 == 0:
            elapsed = time.time() - start_time
            throughput = line_num / elapsed if elapsed > 0 else 0
            print(f"Processed: {line_num:,} lines ({throughput:.0f} lines/sec)", end='\r')
        
        # Limitar a muestra si se especificó
        if SAMPLE_SIZE and line_num > SAMPLE_SIZE:
            break
        
        # Parsear línea
        try:
            tweet = orjson.loads(line)
            
            # Verificar que sea un diccionario (no un número u otro tipo)
            if not isinstance(tweet, dict):
                stats['non_dict_lines'] += 1
                stats['invalid_lines'] += 1
                if len(stats['parse_errors']) < 5:
                    stats['parse_errors'].append({
                        'line': line_num,
                        'error': f'Not a dictionary: {type(tweet).__name__}',
                        'sample': str(tweet)[:100]
                    })
                continue
            
            stats['valid_lines'] += 1
            
            # Guardar algunos tweets de muestra
            if len(stats['sample_tweets']) < 3:
                stats['sample_tweets'].append(tweet)
            
            # Analizar campos críticos
            for field in ['date', 'content', 'user', 'mentionedUsers']:
                if field not in tweet or tweet[field] is None:
                    stats['missing_fields'][field] += 1
            
            # Analizar fecha
            if 'date' in tweet and tweet['date']:
                date_str = str(tweet['date'])
                if 'T' in date_str:
                    stats['date_formats']['ISO-8601'] += 1
                else:
                    stats['date_formats']['OTHER'] += 1
            
            # Analizar contenido
            if 'content' in tweet and tweet['content']:
                content = str(tweet['content'])
                stats['content_lengths'].append(len(content))
                
                # Detectar emojis
                emojis_in_tweet = emoji.emoji_count(content)
                if emojis_in_tweet > 0:
                    stats['tweets_with_emojis'] += 1
                    stats['emoji_count'] += emojis_in_tweet
            
            # Analizar menciones
            if 'mentionedUsers' not in tweet or tweet['mentionedUsers'] is None:
                mentions_stats['null_mentions'] += 1
            elif isinstance(tweet['mentionedUsers'], list):
                num_mentions = len(tweet['mentionedUsers'])
                stats['mentions_distribution'].append(num_mentions)
                
                if num_mentions == 0:
                    mentions_stats['empty_list'] += 1
                    mentions_stats['without_mentions'] += 1
                else:
                    mentions_stats['with_mentions'] += 1
            
            # Verificar user.username
            if 'user' in tweet and isinstance(tweet['user'], dict):
                if 'username' not in tweet['user'] or tweet['user']['username'] is None:
                    stats['missing_fields']['user.username'] += 1
            
        except orjson.JSONDecodeError as e:
            stats['invalid_lines'] += 1
            if len(stats['parse_errors']) < 5:
                stats['parse_errors'].append({
                    'line': line_num,
                    'error': str(e),
                    'sample': line[:100].decode('utf-8', errors='ignore')
                })

processing_time = time.time() - start_time

print(f"\n\nAnalysis completed in {processing_time:.2f} seconds")
print(f"Average throughput: {(stats['valid_lines'] + stats['invalid_lines'])/processing_time:.0f} lines/sec")


  COMPLETE STATISTICAL ANALYSIS

Analyzing all tweets for detailed statistics...

Processed: 110,000 lines (17336 lines/sec)

Analysis completed in 6.76 seconds
Average throughput: 17359 lines/sec


### Análisis estadístico completo del dataset

En esta sección se recorrió el **dataset completo en streaming** para obtener estadísticas detalladas de calidad, contenido y estructura, sin cargar el archivo completo en memoria.

Durante el recorrido se midió el **throughput de procesamiento**, alcanzando aproximadamente **17.3K tweets por segundo**, lo que confirma que el enfoque de lectura línea a línea con `orjson` es adecuado para los enfoques optimizados en memoria.

Este análisis permitió:
- Cuantificar la presencia de campos faltantes en atributos críticos.
- Confirmar el formato de fechas predominante (ISO 8601).
- Medir la distribución de longitudes de texto.
- Evaluar la frecuencia y distribución de menciones (`mentionedUsers`).
- Detectar la presencia y cantidad de emojis en el contenido.
- Identificar líneas inválidas o no conformes (si existieran).

Los resultados validan que el dataset puede procesarse eficientemente en una sola pasada, y sirven como base para definir las estrategias finales de implementación para q1, q2 y q3.


### Resultados: Validación Básica

In [21]:
total_processed = stats['valid_lines'] + stats['invalid_lines']
invalid_pct = (stats['invalid_lines'] / total_processed * 100) if total_processed > 0 else 0

print("="*60)
print("  BASIC VALIDATION")
print("="*60)
print(f"Total processed: {total_processed:,} lines")
print(f"Valid lines: {stats['valid_lines']:,} ({100-invalid_pct:.2f}%)")
print(f"Invalid lines: {stats['invalid_lines']:,} ({invalid_pct:.2f}%)")

if stats.get('non_dict_lines', 0) > 0:
    print(f"  - Non-dictionary JSON: {stats['non_dict_lines']:,}")

print()

if stats['parse_errors']:
    print("Parse errors (first 5):")
    for error in stats['parse_errors']:
        print(f"  Line {error['line']}: {error['error']}")
        print(f"  Sample: {error['sample'][:80]}...")
        print()

  BASIC VALIDATION
Total processed: 117,407 lines
Valid lines: 117,407 (100.00%)
Invalid lines: 0 (0.00%)



### Validación básica del dataset

Se procesaron **117.407 líneas**, todas válidas (100%), sin registros corruptos ni errores de parseo, lo que confirma que el dataset puede procesarse directamente sin una etapa previa de limpieza estructural.


### Resultados: Calidad de Datos (Campos Faltantes)

In [22]:
print("="*60)
print("  DATA QUALITY - MISSING FIELDS")
print("="*60)

if stats['missing_fields']:
    missing_df = pd.DataFrame([
        {
            'Field': field,
            'Missing': count,
            'Percentage': f"{(count/stats['valid_lines']*100):.2f}%"
        }
        for field, count in sorted(stats['missing_fields'].items(), key=lambda x: x[1], reverse=True)
    ])
    
    print(missing_df.to_string(index=False))
else:
    print("No missing fields detected")

print()

  DATA QUALITY - MISSING FIELDS
         Field  Missing Percentage
mentionedUsers    79373     67.60%



### Calidad de datos, campos faltantes

El campo **`mentionedUsers`** está ausente en aproximadamente **67.6%** de los tweets. Esto implica que, para **q3**, la lógica debe tratar explícitamente este campo como opcional y considerar que la ausencia de menciones es el caso dominante. Para **q1** y **q2**, este faltante no afecta el cálculo, ya que no dependen de este atributo.


## 3. Análisis de Campos Relevantes para las Preguntas

### 3.1. Campo `date` (Relevante para Q1)

In [23]:
print("="*60)
print("  DATE ANALYSIS (Q1)")
print("="*60)

if stats['date_formats']:
    print("\nDetected formats:")
    for fmt, count in stats['date_formats'].most_common():
        pct = (count / stats['valid_lines'] * 100) if stats['valid_lines'] > 0 else 0
        print(f"  {fmt}: {count:,} ({pct:.2f}%)")

# Mostrar ejemplo de fecha
if stats['sample_tweets'] and 'date' in stats['sample_tweets'][0]:
    print(f"\nExample date: {stats['sample_tweets'][0]['date']}")

print(f"\nTweets without date: {stats['missing_fields'].get('date', 0):,}")
print()

  DATE ANALYSIS (Q1)

Detected formats:
  ISO-8601: 117,407 (100.00%)

Example date: 2021-02-24T09:23:35+00:00

Tweets without date: 0



### Análisis de fechas (Q1)

El **100%** de los tweets contiene el campo `date` en formato **ISO 8601**, sin registros faltantes. Esto permite parsear y agrupar fechas de forma directa y eficiente para **q1**, sin necesidad de normalización adicional ni manejo de casos borde por ausencia de fecha.


### 3.2. Campo `content` (Relevante para Q2 - Emojis)

In [24]:
print("="*60)
print("  CONTENT ANALYSIS (Q2)")
print("="*60)

if stats['content_lengths']:
    content_series = pd.Series(stats['content_lengths'])
    
    print("\nTweet length statistics:")
    print(f"  Minimum: {content_series.min():,} chars")
    print(f"  P50 (median): {content_series.quantile(0.5):.0f} chars")
    print(f"  P95: {content_series.quantile(0.95):.0f} chars")
    print(f"  P99: {content_series.quantile(0.99):.0f} chars")
    print(f"  Maximum: {content_series.max():,} chars")

print(f"\nTweets without content: {stats['missing_fields'].get('content', 0):,}")

print("\nEmoji Analysis:")
pct_with_emojis = (stats['tweets_with_emojis'] / stats['valid_lines'] * 100) if stats['valid_lines'] > 0 else 0
print(f"  Tweets with emojis: {stats['tweets_with_emojis']:,} ({pct_with_emojis:.2f}%)")
print(f"  Total emojis detected: {stats['emoji_count']:,}")
if stats['tweets_with_emojis'] > 0:
    avg_emojis = stats['emoji_count'] / stats['tweets_with_emojis']
    print(f"  Average emojis per tweet (with emojis): {avg_emojis:.2f}")

# Mostrar ejemplo de contenido
if stats['sample_tweets'] and 'content' in stats['sample_tweets'][0]:
    content_sample = stats['sample_tweets'][0]['content'][:150]
    print(f"\nExample content: {content_sample}...")

print()

  CONTENT ANALYSIS (Q2)

Tweet length statistics:
  Minimum: 15 chars
  P50 (median): 128 chars
  P95: 298 chars
  P99: 312 chars
  Maximum: 962 chars

Tweets without content: 0

Emoji Analysis:
  Tweets with emojis: 16,874 (14.37%)
  Total emojis detected: 42,922
  Average emojis per tweet (with emojis): 2.54

Example content: The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. 

@narendramodi @D...



### Análisis de contenido y emojis (Q2)

El contenido textual es consistente y está presente en el **100%** de los tweets, con longitudes moderadas (mediana ≈128 caracteres), lo que permite procesarlo eficientemente en streaming. Aproximadamente **14.4%** de los tweets contiene emojis, con un promedio de **2.5 emojis por tweet** cuando están presentes. Estos resultados indican que **q2** puede resolverse sin preprocesamiento adicional, aplicando detección de emojis directamente sobre el campo `content`.


### 3.3. Campo `mentionedUsers` (Relevante para Q3)

In [25]:
print("="*60)
print("  MENTIONS ANALYSIS (Q3)")
print("="*60)

total = stats['valid_lines']

print("\nMentions distribution:")
print(f"  With mentions: {mentions_stats['with_mentions']:,} ({mentions_stats['with_mentions']/total*100:.2f}%)")
print(f"  Without mentions: {mentions_stats['without_mentions']:,} ({mentions_stats['without_mentions']/total*100:.2f}%)")
print(f"  Null field: {mentions_stats['null_mentions']:,} ({mentions_stats['null_mentions']/total*100:.2f}%)")
print(f"  Empty list: {mentions_stats['empty_list']:,} ({mentions_stats['empty_list']/total*100:.2f}%)")

if stats['mentions_distribution']:
    mentions_series = pd.Series(stats['mentions_distribution'])
    
    print("\nMentions per tweet statistics:")
    print(f"  Average: {mentions_series.mean():.2f} mentions/tweet")
    print(f"  Maximum: {mentions_series.max()} mentions in a tweet")
    
    print("\nMentions count distribution:")
    mention_counts = mentions_series.value_counts().sort_index().head(10)
    for num_mentions, count in mention_counts.items():
        pct = (count / len(mentions_series) * 100)
        print(f"  {num_mentions} mentions: {count:,} tweets ({pct:.2f}%)")

# Mostrar ejemplo de menciones
if stats['sample_tweets']:
    for tweet in stats['sample_tweets']:
        if 'mentionedUsers' in tweet and tweet['mentionedUsers'] and len(tweet['mentionedUsers']) > 0:
            print(f"\nExample mentions: {tweet['mentionedUsers'][:3]}")
            break

print()

  MENTIONS ANALYSIS (Q3)

Mentions distribution:
  With mentions: 38,034 (32.40%)
  Without mentions: 0 (0.00%)
  Null field: 79,373 (67.60%)
  Empty list: 0 (0.00%)

Mentions per tweet statistics:
  Average: 2.72 mentions/tweet
  Maximum: 50 mentions in a tweet

Mentions count distribution:
  1 mentions: 19,422 tweets (51.06%)
  2 mentions: 7,920 tweets (20.82%)
  3 mentions: 3,251 tweets (8.55%)
  4 mentions: 1,878 tweets (4.94%)
  5 mentions: 1,059 tweets (2.78%)
  6 mentions: 971 tweets (2.55%)
  7 mentions: 571 tweets (1.50%)
  8 mentions: 518 tweets (1.36%)
  9 mentions: 540 tweets (1.42%)
  10 mentions: 362 tweets (0.95%)

Example mentions: [{'username': 'narendramodi', 'displayname': 'Narendra Modi', 'id': 18839785, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protect

### Análisis de menciones (Q3)

El **67.6%** de los tweets no contiene el campo `mentionedUsers`, mientras que el **32.4%** sí incluye menciones. Cuando existen, las menciones son moderadas (promedio ≈2.7 por tweet), aunque hay casos extremos con hasta 50 menciones. Esto confirma que **q3** debe tratar `mentionedUsers` como campo opcional y puede calcular la influencia de usuarios eficientemente iterando solo sobre los tweets que contienen menciones, sin necesidad de analizar el texto completo.


## 4. Estructura de un Tweet (Ejemplo Real)

In [26]:
import json

if stats['sample_tweets']:
    print("="*60)
    print("  COMPLETE TWEET EXAMPLE")
    print("="*60)
    
    sample = stats['sample_tweets'][0]
    
    # Mostrar solo campos relevantes para el challenge
    relevant_fields = {
        'date': sample.get('date'),
        'content': sample.get('content', '')[:200] + '...' if sample.get('content') else None,
        'user': {
            'username': sample.get('user', {}).get('username') if isinstance(sample.get('user'), dict) else None
        },
        'mentionedUsers': sample.get('mentionedUsers'),
    }
    
    print(json.dumps(relevant_fields, indent=2, ensure_ascii=False))
    print()

  COMPLETE TWEET EXAMPLE
{
  "date": "2021-02-24T09:23:35+00:00",
  "content": "The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#...",
  "user": {
    "username": "ArjunSinghPanam"
  },
  "mentionedUsers": [
    {
      "username": "narendramodi",
      "displayname": "Narendra Modi",
      "id": 18839785,
      "description": null,
      "rawDescription": null,
      "descriptionUrls": null,
      "verified": null,
      "created": null,
      "followersCount": null,
      "friendsCount": null,
      "statusesCount": null,
      "favouritesCount": null,
      "listedCount": null,
      "mediaCount": null,
      "location": null,
      "protected": null,
      "linkUrl": null,
      "linkTcourl": null,
      "profileImageUrl": null,
      "profileBannerUrl": null,
      "url": "https://twitter.com/narendramodi"
    },
    {
      "u

## 5. Performance y Estimaciones

Estos datos son críticos para decidir entre enfoques time-optimized vs memory-optimized.

In [27]:
print("="*60)
print("  PERFORMANCE AND ESTIMATIONS")
print("="*60)

throughput = (stats['valid_lines'] + stats['invalid_lines']) / processing_time

print(f"\nComplete processing time: {processing_time:.2f} seconds")
print(f"Streaming read throughput: {throughput:.0f} lines/second")
print(f"Throughput: ~{throughput*60:.0f} lines/minute")

# Estimaciones
if total_lines > (stats['valid_lines'] + stats['invalid_lines']):
    estimated_time = total_lines / throughput
    print(f"\nEstimated time for complete dataset: {estimated_time:.2f} seconds (~{estimated_time/60:.1f} minutes)")

print("\nImplications for time vs memory:")
print("  - Streaming is viable: high throughput")
print("  - Memory approach: streaming line by line")
print("  - Time approach: load in memory with pandas/polars if sufficient RAM")
print()

  PERFORMANCE AND ESTIMATIONS

Complete processing time: 6.76 seconds
Streaming read throughput: 17359 lines/second
Throughput: ~1041555 lines/minute

Implications for time vs memory:
  - Streaming is viable: high throughput
  - Memory approach: streaming line by line
  - Time approach: load in memory with pandas/polars if sufficient RAM



### Rendimiento y estimaciones

El dataset completo puede procesarse en **~6.8 segundos** usando lectura en streaming, con un throughput de **~17.3K tweets por segundo**. Esto confirma que el enfoque optimizado en memoria puede resolverse eficientemente en una sola pasada. Dado este rendimiento y la disponibilidad de RAM, el enfoque optimizado en tiempo puede cargar el dataset en memoria (por ejemplo con pandas o polars) para acelerar operaciones agregadas sin riesgo significativo.


## 6. Casos Borde y Decisiones Técnicas

### Decisiones basadas en evidencia:

## 7. Conclusiones y Decisiones Técnicas Finales

### Resumen Ejecutivo

El profiling exhaustivo del dataset reveló **117.407 tweets** (389 MB) con una calidad de datos superior a la esperada:

1. **Dataset completamente válido**: 100% de líneas parseables, 0 errores de JSON
2. **Campos críticos completos**: `date`, `content` y `user.username` presentes en el 100% de los registros
3. **Esquema consistente**: 21 campos identificados con tipos de datos estables
4. **Performance excelente**: 17.3K tweets/segundo en lectura streaming (6.8 segundos para procesamiento completo)

### Hallazgos Clave por Pregunta

#### Q1: Top 10 fechas con más tweets

**Datos confirmados:**
- **100%** de tweets tiene campo `date` en formato ISO 8601
- **100%** de tweets tiene `user.username`
- **0** registros requieren limpieza o normalización

**Decisión técnica:**
- Parsear con `datetime.fromisoformat()` sin validaciones adicionales
- Extraer solo la fecha (ignorar hora) para agrupar por día
- Contar usuarios únicos por fecha usando estructuras simples (Counter, defaultdict)

#### Q2: Top 10 emojis más usados

**Datos confirmados:**
- **100%** de tweets tiene campo `content`
- **14.4%** de tweets contiene al menos un emoji (16.874 tweets)
- **42.922 emojis totales**, promedio de 2.5 por tweet con emojis

**Decisión técnica:**
- Aplicar librería `emoji` directamente sobre `content` sin preprocesamiento
- Usar `emoji.emoji_list()` para manejar emojis compuestos correctamente
- No extraer de campos alternativos (renderedContent), `content` es suficiente

#### Q3: Top 10 usuarios más influyentes (por menciones recibidas)

**Datos confirmados:**
- **67.6%** de tweets NO tiene `mentionedUsers` (campo null)
- **32.4%** de tweets SÍ tiene menciones (38.034 tweets)
- Promedio de **2.7 menciones/tweet** cuando existen
- Casos extremos: hasta **50 menciones** en un solo tweet

**Decisión técnica:**
- Tratar `mentionedUsers == null` como ausencia de menciones (no como error)
- Iterar solo sobre tweets con menciones (saltar 67.6% del dataset)
- Extraer `username` del objeto estructurado en la lista
- No parsear texto, confiar en campo estructurado

### Performance y Estrategia de Implementación

**Enfoque optimizado en TIEMPO:**
- Cargar dataset completo en memoria con **Polars**
- Aprovechar operaciones vectorizadas y paralelización automática
- Dataset cabe holgadamente en RAM moderna (389 MB)
- Tiempo esperado: **< 2 segundos** por pregunta

**Enfoque optimizado en MEMORIA:**
- Streaming línea por línea con **orjson**
- Estructuras minimalistas: `Counter`, `defaultdict`, `heapq`
- Una sola pasada por el dataset, sin carga completa
- Tiempo esperado: **6-8 segundos** por pregunta

### Casos Borde Confirmados

| Caso | Frecuencia | Estrategia |
|------|-----------|-----------|
| Tweets sin `date` | 0 (0%) | No requiere manejo |
| Tweets sin `content` | 0 (0%) | No requiere manejo |
| Tweets sin `user.username` | 0 (0%) | No requiere manejo |
| Tweets sin `mentionedUsers` | 79.373 (67.6%) | Saltar en Q3 |
| Líneas JSON inválidas | 0 (0%) | No requiere manejo |

### Decisiones de Diseño Finales

**✅ Haremos:**
- Parseo directo sin validación defensiva excesiva (calidad confirmada)
- Streaming para enfoque memory, carga completa para enfoque time
- Uso de `mentionedUsers` estructurado (no parsing de texto)
- Manejo explícito de `mentionedUsers == null` en Q3

**❌ NO haremos:**
- Limpieza o normalización de datos (no es necesaria)
- Imputación de campos faltantes (no existen casos críticos)
- Extracción de menciones desde texto (campo estructurado disponible)
- Validación exhaustiva línea por línea (100% de datos válidos)

### Próximos Pasos

1. ✅ **Profiling completado** → Decisiones técnicas documentadas
2. → **Implementar Q1**: Top fechas + usuarios por fecha
3. → **Implementar Q2**: Top emojis
4. → **Implementar Q3**: Top usuarios influyentes

**Este profiling elimina incertidumbre técnica y permite implementación directa sin iteraciones exploratorias.**